In [1]:
!pip install yfinance
!pip install bs4
!pip install nbformat

     ---------------------------------------- 0.0/939.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/939.5 kB ? eta -:--:--
     ---------------------------------------- 10.2/939.5 kB ? eta -:--:--
     - ----------------------------------- 30.7/939.5 kB 330.3 kB/s eta 0:00:03
     - ----------------------------------- 41.0/939.5 kB 245.8 kB/s eta 0:00:04
     -- ---------------------------------- 61.4/939.5 kB 328.2 kB/s eta 0:00:03
     -- ---------------------------------- 71.7/939.5 kB 328.6 kB/s eta 0:00:03
     ---- ------------------------------- 122.9/939.5 kB 423.5 kB/s eta 0:00:02
     ----- ------------------------------ 153.6/939.5 kB 482.7 kB/s eta 0:00:02
     ------- ---------------------------- 204.8/939.5 kB 541.9 kB/s eta 0:00:02
     ---------- ------------------------- 276.5/939.5 kB 655.8 kB/s eta 0:00:02
     ------------ ----------------------- 337.9/939.5 kB 723.4 kB/s eta 0:00:01
     ---------------- ------------------- 440.3/939.5 kB 860

In [6]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [7]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [8]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [9]:
Tesla = yf.Ticker("TSLA")

In [10]:
tesla_data = Tesla.history(period = "max")
print(tesla_data.head())

                               Open      High       Low     Close     Volume  \
Date                                                                           
2010-06-29 00:00:00-04:00  1.266667  1.666667  1.169333  1.592667  281494500   
2010-06-30 00:00:00-04:00  1.719333  2.028000  1.553333  1.588667  257806500   
2010-07-01 00:00:00-04:00  1.666667  1.728000  1.351333  1.464000  123282000   
2010-07-02 00:00:00-04:00  1.533333  1.540000  1.247333  1.280000   77097000   
2010-07-06 00:00:00-04:00  1.333333  1.333333  1.055333  1.074000  103003500   

                           Dividends  Stock Splits  
Date                                                
2010-06-29 00:00:00-04:00        0.0           0.0  
2010-06-30 00:00:00-04:00        0.0           0.0  
2010-07-01 00:00:00-04:00        0.0           0.0  
2010-07-02 00:00:00-04:00        0.0           0.0  
2010-07-06 00:00:00-04:00        0.0           0.0  


In [11]:
tesla_data.reset_index(inplace=True)
print(tesla_data.head())

                       Date      Open      High       Low     Close  \
0 2010-06-29 00:00:00-04:00  1.266667  1.666667  1.169333  1.592667   
1 2010-06-30 00:00:00-04:00  1.719333  2.028000  1.553333  1.588667   
2 2010-07-01 00:00:00-04:00  1.666667  1.728000  1.351333  1.464000   
3 2010-07-02 00:00:00-04:00  1.533333  1.540000  1.247333  1.280000   
4 2010-07-06 00:00:00-04:00  1.333333  1.333333  1.055333  1.074000   

      Volume  Dividends  Stock Splits  
0  281494500        0.0           0.0  
1  257806500        0.0           0.0  
2  123282000        0.0           0.0  
3   77097000        0.0           0.0  
4  103003500        0.0           0.0  


In [12]:
# Using Webscraping to extract Tesla Revenue Data
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
response = requests.get(url)
html_data = response.text

In [14]:
# Parse the HTML data using BeautifulSoup
soup = BeautifulSoup(html_data, 'html.parser')

In [15]:
# Use read_html to extract tables from the webpage
tables = pd.read_html(url)

tesla_revenue = tables[0]

tesla_revenue.columns = ['Date', 'Revenue']

In [16]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")

In [17]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [18]:
print(tesla_revenue.tail())

    Date Revenue
8   2013  $2,013
9   2012    $413
10  2011    $204
11  2010    $117
12  2009    $112


In [19]:
# Use yfinance to extract Stock Data
GameStop = yf.Ticker("GME")

In [20]:
gme_data = GameStop.history(period = "max")
print(gme_data.head())

                               Open      High       Low     Close    Volume  \
Date                                                                          
2002-02-13 00:00:00-05:00  1.620128  1.693350  1.603296  1.691666  76216000   
2002-02-14 00:00:00-05:00  1.712707  1.716074  1.670626  1.683250  11021600   
2002-02-15 00:00:00-05:00  1.683251  1.687459  1.658002  1.674834   8389600   
2002-02-19 00:00:00-05:00  1.666418  1.666418  1.578047  1.607504   7410400   
2002-02-20 00:00:00-05:00  1.615920  1.662210  1.603296  1.662210   6892800   

                           Dividends  Stock Splits  
Date                                                
2002-02-13 00:00:00-05:00        0.0           0.0  
2002-02-14 00:00:00-05:00        0.0           0.0  
2002-02-15 00:00:00-05:00        0.0           0.0  
2002-02-19 00:00:00-05:00        0.0           0.0  
2002-02-20 00:00:00-05:00        0.0           0.0  


In [21]:
gme_data.reset_index(inplace=True)
print(gme_data.head())

                       Date      Open      High       Low     Close    Volume  \
0 2002-02-13 00:00:00-05:00  1.620128  1.693350  1.603296  1.691666  76216000   
1 2002-02-14 00:00:00-05:00  1.712707  1.716074  1.670626  1.683250  11021600   
2 2002-02-15 00:00:00-05:00  1.683251  1.687459  1.658002  1.674834   8389600   
3 2002-02-19 00:00:00-05:00  1.666418  1.666418  1.578047  1.607504   7410400   
4 2002-02-20 00:00:00-05:00  1.615920  1.662210  1.603296  1.662210   6892800   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0  


In [22]:
# Use Webscraping to extract GME Revenue Data
url = ' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html. '
response = requests.get(url)
html_data_2 = response.text

In [24]:
soup = BeautifulSoup(html_data_2, 'html.parser')

Using BeautifulSoup or the read_html function extract the table with GameStop Revenue and store it into a dataframe named gme_revenue. The dataframe should have columns Date and Revenue. Make sure the comma and dollar sign is removed from the Revenue column.

In [28]:
# Check if the second tbody exists
tbody_elements = soup.find_all('tbody')
if len(tbody_elements) > 1:
    gme_tbody = tbody_elements[1]
else:
    print("The second tbody element does not exist.")
    gme_tbody = None

# If tbody was found, proceed with data extraction
if gme_tbody:
    rows = gme_tbody.find_all('tr')
    data = []

    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        if cols:
            data.append(cols)

    # Create DataFrame
    gme_revenue = pd.DataFrame(data, columns=['Date', 'Revenue'])

    # Remove commas and dollar signs from Revenue and convert to numeric
    gme_revenue['Revenue'] = gme_revenue['Revenue'].replace({'\$': '', ',': ''}, regex=True)
    gme_revenue['Revenue'] = pd.to_numeric(gme_revenue['Revenue'], errors='coerce')

    # Print the tail of the DataFrame
    print(gme_revenue.tail())
else:
    print("No valid tbody found to extract data.")

The second tbody element does not exist.
No valid tbody found to extract data.


# Plot Tesla Stock Graph

In [31]:
def make_graph(tesla_data, tesla_revenue, Tesla):
    # Ensure Date columns are datetime and handle invalid dates
    tesla_data['Date'] = pd.to_datetime(tesla_data['Date'], errors='coerce')
    tesla_revenue['Date'] = pd.to_datetime(tesla_revenue['Date'], errors='coerce')

    # Filter data for specific date ranges
    stock_data_specific = tesla_data[tesla_data.Date <= '2021-06-14']
    revenue_data_specific = tesla_revenue[tesla_revenue.Date <= '2021-04-30']

    # Create subplots
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing=0.3)

    # Add stock price trace
    fig.add_trace(go.Scatter(x=stock_data_specific['Date'], y=stock_data_specific['Close'].astype("float"), name="Share Price"), row=1, col=1)

    # Add revenue trace
    fig.add_trace(go.Scatter(x=revenue_data_specific['Date'], y=revenue_data_specific['Revenue'].astype("float"), name="Revenue"), row=2, col=1)

    # Update axis titles
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)

    # Layout adjustments
    fig.update_layout(showlegend=False, height=900, title=Tesla, xaxis_rangeslider_visible=True)

    # Show the figure
    fig.show()


#  Plot GameStop Stock Graph

In [30]:
def make_graph(gme_data, gme_revenue, GameStop):
    # Ensure Date columns are datetime and handle invalid dates
    gme_data['Date'] = pd.to_datetime(gme_data['Date'], errors='coerce')
    gme_revenue['Date'] = pd.to_datetime(gme_revenue['Date'], errors='coerce')

    # Filter data for specific date ranges
    stock_data_specific = gme_data[gme_data.Date <= '2021-06-14']
    revenue_data_specific = gme_revenue[gme_revenue.Date <= '2021-04-30']

    # Create subplots
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing=0.3)

    # Add stock price trace
    fig.add_trace(go.Scatter(x=stock_data_specific['Date'], y=stock_data_specific['Close'].astype("float"), name="Share Price"), row=1, col=1)

    # Add revenue trace
    fig.add_trace(go.Scatter(x=revenue_data_specific['Date'], y=revenue_data_specific['Revenue'].astype("float"), name="Revenue"), row=2, col=1)

    # Update axis titles
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)

    # Layout adjustments
    fig.update_layout(showlegend=False, height=900, title=GameStop, xaxis_rangeslider_visible=True)

    # Show the figure
    fig.show()
